In [2]:
# David Ouyang 10/2/2019

# Notebook which iterates through a folder, including subfolders, 
# and convert DICOM files to AVI files of a defined size (natively 112 x 112)

import re
import os, os.path
from os.path import splitext
import pydicom as dicom
import numpy as np
from pydicom.uid import UID, generate_uid
import shutil
from multiprocessing import dummy as multiprocessing
import time
import subprocess
import datetime
from datetime import date
import sys
import cv2
#from scipy.misc import imread
import matplotlib.pyplot as plt
import sys
from shutil import copy
import math



In [1]:
# Dependencies you might need to run code
# Commonly missing

#!pip install pydicom
#!pip install opencv-python


In [4]:
def mask(output):
    dimension = output.shape[0]
    
    # Mask pixels outside of scanning sector
    m1, m2 = np.meshgrid(np.arange(dimension), np.arange(dimension))
    

    mask = ((m1+m2)>int(dimension/2) + int(dimension/10)) 
    mask *=  ((m1-m2)<int(dimension/2) + int(dimension/10))
    mask = np.reshape(mask, (dimension, dimension)).astype(np.int8)
    maskedImage = cv2.bitwise_and(output, output, mask = mask)
    
    #print(maskedImage.shape)
    
    return maskedImage



In [5]:
# Code to preprocess DICOM videos.
def makeVideo(fileToProcess, destinationFolder):
    try:
        fileName = fileToProcess.split('\\')[-1] #\\ if windows, / if on mac or sherlock
                                                 #hex(abs(hash(fileToProcess.split('/')[-1]))).upper()

        if not os.path.isdir(os.path.join(destinationFolder,fileName)):

            dataset = dicom.dcmread(fileToProcess, force=True)
            testarray = dataset.pixel_array

            frame0 = testarray[0]
            mean = np.mean(frame0, axis=1)
            mean = np.mean(mean, axis=1)
            yCrop = np.where(mean<1)[0][0]
            testarray = testarray[:, yCrop:, :, :]

            bias = int(np.abs(testarray.shape[2] - testarray.shape[1])/2)
            if bias>0:
                if testarray.shape[1] < testarray.shape[2]:
                    testarray = testarray[:, :, bias:-bias, :]
                else:
                    testarray = testarray[:, bias:-bias, :, :]


            print(testarray.shape)
            frames,height,width,channels = testarray.shape

            fps = 30

            try:
                fps = dataset[(0x18, 0x40)].value
            except:
                print("couldn't find frame rate, default to 30")

            fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
            video_filename = os.path.join(destinationFolder, fileName + '.avi')
            out = cv2.VideoWriter(video_filename, fourcc, fps, cropSize)


            for i in range(frames):

                outputA = testarray[i,:,:,0]
                smallOutput = outputA[int(height/10):(height - int(height/10)), int(height/10):(height - int(height/10))]

                # Resize image
                output = cv2.resize(smallOutput, cropSize, interpolation = cv2.INTER_CUBIC)

                finaloutput = mask(output)


                finaloutput = cv2.merge([finaloutput,finaloutput,finaloutput])
                out.write(finaloutput)

            out.release()

        else:
            print(fileName,"hasAlreadyBeenProcessed")
    except:
        print("something filed, not sure what, have to debug", fileName)
    return 0


In [1]:
# Code to precprocess .avi videos. Preprocessing of  '<name>.avi' files. The echonet segmentation only accepts 112x112 videos. So performing a crop, resize and flip on the original videos.
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mayo_scripts.process_external_james import segment_cardiac_echo

def convertVideo(input_file, destination_file, final_size=(112, 112), crop_size=(380, 320), fps=20):
    try:
        fileName = os.path.basename(input_file)
        output_file = destination_file

        cap = cv2.VideoCapture(input_file)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        print(frame_height, frame_width)
        v = np.zeros((frame_count, frame_height, frame_width, 3), np.uint8)

        for count in range(frame_count):
            ret, frame = cap.read()
            if not ret:
                raise ValueError("Failed to load frame #{} of {}.".format(count, input_file))

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            contour_mask_single_channel = segment_cardiac_echo(frame_gray)
            contour_mask = np.zeros_like(frame)
            # print(frame.shape)
            for channel in range(frame.shape[2]):
                contour_mask[:, :, channel] = contour_mask_single_channel

            # Apply the contour mask to each channel of the input frame
            frame_crop = frame * contour_mask
            v[count, :, :, :] = frame_crop

        frame_h = int(frame.shape[0] * 0.13)
        frame_w = int(frame.shape[1] * 0.21)
        # Crop the middle region
        crop_x = int((frame_width - crop_size[0]) / 2)
        crop_y = int((frame_height - crop_size[1]) / 2)
        # cropped_video = v[:, crop_y:crop_y+crop_size[1], crop_x:crop_x+crop_size[0], :]
        cropped_video = v[:, frame_h:-frame_h, frame_w:-frame_w, :]

        # Resize cropped video frames to the final size (112x112)
        resized_video = np.zeros((frame_count, final_size[1], final_size[0], 3), np.uint8)
        for count in range(frame_count):
            resized_frame = cv2.resize(cropped_video[count, :, :, :], final_size, interpolation=cv2.INTER_CUBIC)
            resized_video[count, :, :, :] = resized_frame

        # Transpose dimensions to (channels=3, frames, height, width)
        resized_video = resized_video.transpose((3, 0, 1, 2))

        # Save the converted video with 'M','J','P','G' encoding
        fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
        out = cv2.VideoWriter(output_file, fourcc, fps, final_size)
        for count in range(frame_count):
            frame = resized_video[:, count, :, :].transpose((1, 2, 0))
            out.write(frame)

        cap.release()
        out.release()
        cv2.destroyAllWindows()

    except Exception as e:
        print("Something failed, not sure what, have to debug:", fileName)
        print(e)

    return 0


In [2]:
AllA4cNames = "/mnt/storage/Anonymized_echo_AP4/abnormals"
output_folder = "/home/lalith/scratch/converted_echo_abnormal_AP4"

count = 10
    
cropSize = (112,112)


for content in os.listdir(AllA4cNames):
    VideoPath = os.path.join(AllA4cNames, content)

    print(count, content)

    # if not os.path.exists(os.path.join(output_folder, 'converted_'+content)):
    convertVideo(VideoPath, os.path.join(output_folder, 'converted_'+content))
        # makeVideo(VideoPath, destinationFolder)
    # else:
    #     print("Already did this file", VideoPath)


# print(len(AllA4cFilenames))

10 R468_AP4.avi
434 636
10 R460_AP4.avi
434 636
10 R204_AP4.avi
434 636
10 R502_AP4.avi
434 636
10 R205_AP4.avi
434 636
10 R473_AP4.avi
434 636
10 R582_AP4.avi
434 636
10 R233_AP4.avi
434 636
10 R489_AP4.avi
434 636
10 R186_AP4.avi
434 636
10 R471_AP4.avi
434 636
10 R461_AP4.avi
434 636
10 R169_AP4.avi
434 636
10 R495_AP4.avi
434 636
10 R526_AP4.avi
434 636
10 R358_AP4.avi
434 636
10 R425_AP4.avi
434 636
10 R499_AP4.avi
434 636
10 R510_AP4.avi
434 636
10 R525_AP4.avi
434 636
10 R192_AP4.avi
434 636
10 R408_AP4.avi
434 636
10 R506_AP4.avi
434 636
10 R338_AP4.avi
434 636
